In [2]:
import os
import sys
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import time
import calendar
from datetime import datetime, date, timedelta
import pickle


In [3]:
#open the df_bitfinex_volume.csv file
df_bitfinex_volume = pd.read_csv("df_bitfinex_volume.csv")
df_bitfinex_volume.index =df_bitfinex_volume['Date']
df_bitfinex_volume = df_bitfinex_volume.drop(['Date'], axis=1)
df_bitfinex_volume

,Daily Volume
Date,
2013-03-31,5224.401313
2013-04-01,8376.527478
2013-04-02,12996.245072
2013-04-03,6981.668305
2013-04-04,5234.882203
...,...
2023-07-11,1636.401373
2023-07-12,3763.115501
2023-07-13,2982.608992


In [4]:
# open the df_price_btc_usd_mtgox_daily file
df_price_btc_usd_mtgox_daily = pd.read_csv("/home/moises/Data_BTC_thesis/df_price_btc_usd_agregate.csv")
df_price_btc_usd_mtgox_daily_svar = df_price_btc_usd_mtgox_daily[df_price_btc_usd_mtgox_daily["Date"]>="2013-03-31"] 
df_price_btc_usd_mtgox_daily_svar.index = df_price_btc_usd_mtgox_daily_svar['Date']
del df_price_btc_usd_mtgox_daily_svar['Date']
df_price_btc_usd_mtgox_daily_svar


,Close
Date,
2013-03-31,92.743611
2013-04-01,100.409370
2013-04-02,108.063618
2013-04-03,128.261101
2013-04-04,132.155428
...,...
2023-07-27,29312.836826
2023-07-28,29358.839335
2023-07-29,29268.279610


In [22]:
#get df_blockstats_daily_mean.csv
df_blockstats_daily_mean = pd.read_csv("/home/moises/Data_BTC_thesis/df_blockstats_daily_mean.csv")
df_blockstats_daily_mean.columns

Index(['GMT', 'index', 'avgfee', 'avgfeerate', 'avgtxsize', 'height', 'ins',
       'maxfee', 'maxfeerate', 'maxtxsize', 'medianfee', 'mediantime',
       'mediantxsize', 'minfee', 'minfeerate', 'mintxsize', 'outs', 'subsidy',
       'swtotal_size', 'swtotal_weight', 'swtxs', 'time', 'total_out',
       'total_size', 'total_weight', 'totalfee', 'txs', 'utxo_increase',
       'utxo_size_inc', 'utxo_increase_actual', 'utxo_size_inc_actual', 'year',
       'avgfee_btc', 'maxfee_btc', 'minfee_btc', 'subsidy_btc',
       'blockhash_decimal', 'Close', 'Date', 'avgfee_USD', 'maxfee_USD',
       'minfee_USD'],
      dtype='object')

In [23]:
df_blockstats_daily_mean["total_out_usd"] =(df_blockstats_daily_mean["total_out"] + df_blockstats_daily_mean["totalfee"] + df_blockstats_daily_mean["subsidy"])

In [21]:
#df_blockstats_daily_mean = df_blockstats_daily_mean[df_blockstats_daily_mean["Date"]>="2013-03-31"]
df_blockstats_daily_mean

,GMT,index,avgfee,avgfeerate,avgtxsize,height,ins,maxfee,maxfeerate,maxtxsize,...,maxfee_btc,minfee_btc,subsidy_btc,blockhash_decimal,Close,Date,avgfee_USD,maxfee_USD,minfee_USD,total_out_usd
1542,2013-03-31,879.875,97333.220238,163.267857,830.791667,228922.541667,707.571429,7.775612e+06,3338.738095,16464.940476,...,0.077756,5.952381e-07,25.00,2.082020e+60,92.743611,2013-03-31,0.090270,7.211384,0.000055,9.171621e+11
1543,2013-04-01,105.500,80666.772727,158.116162,570.424242,229106.500000,650.797980,2.725213e+06,5080.934343,15467.085859,...,0.027252,1.515152e-06,25.00,2.113253e+60,100.409370,2013-04-01,0.080997,2.736369,0.000152,7.318517e+11
1544,2013-04-02,295.000,81245.955801,161.950276,528.209945,229296.000000,811.618785,8.303811e+06,7573.016575,16302.441989,...,0.083038,1.104972e-06,25.00,1.948098e+60,108.063618,2013-04-02,0.087797,8.973399,0.000119,1.430121e+12
1545,2013-04-03,473.000,95721.051429,194.994286,533.354286,229474.000000,849.680000,1.400018e+07,10347.594286,14619.680000,...,0.140002,0.000000e+00,25.00,2.049811e+60,128.261101,2013-04-03,0.122773,17.956781,0.000000,1.355043e+12
1546,2013-04-04,637.500,98543.233766,192.428571,571.006494,229638.500000,1032.461039,1.963353e+07,6570.993506,18351.681818,...,0.196335,3.896104e-06,25.00,2.109398e+60,132.155428,2013-04-04,0.130230,25.946782,0.000515,1.488745e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312,2023-07-27,479.500,5532.333333,14.608696,605.963768,800480.500000,6846.550725,5.998281e+05,621.224638,83826.811594,...,0.005998,5.944783e-06,6.25,2.398011e+53,29312.836826,2023-07-27,1.621684,175.826645,0.174258,5.359591e+11
5313,2023-07-28,618.500,5386.328571,14.085714,564.642857,800619.500000,6824.942857,5.802783e+05,531.842857,80769.614286,...,0.005803,5.869643e-06,6.25,2.588662e+53,29358.839335,2023-07-28,1.581364,170.362980,0.172326,5.618714e+11
5314,2023-07-29,754.500,4496.022727,11.750000,569.560606,800755.500000,6682.159091,4.889126e+05,449.606061,82856.401515,...,0.004889,5.975227e-06,6.25,2.491290e+53,29268.279610,2023-07-29,1.315909,143.096298,0.174885,4.065630e+11
5315,2023-07-30,904.500,3355.285714,9.702381,520.672619,800905.500000,6601.208333,4.179448e+05,412.202381,77184.083333,...,0.004179,5.465417e-06,6.25,2.656606e+53,29223.472673,2023-07-30,0.980531,122.137994,0.159718,3.725155e+11
